In [4]:
import warnings, inspect, osconfeed

DB_NAME = 'data/schedule2_db'
CONFERENCE = 'conference.115'

        


ModuleNotFoundError: No module named 'osconfeed'

In [5]:
class MissingDatabaseError(RuntimeError) : 
    """필요한 데이터베이스가 설정되어 있지 않을 때 발생함."""
  
class Record : 
    def __init__(self, **kwargs) : 
        self.__dict__.update(kwargs)
        
    def __eq__(self, other) : 
        if isinstance(ohter, Record) : 
            return self.__dict__ == other.__dict__
        
        else : 
            return NotImplemented

class DbRecord(Record) : 
    
    __db = None
    
    @staticmethod
    def set_db(db) : 
        DbRecord.__db = db
        
    @staticmethod 
    def get_db() : 
        return DbRecord.__db
    
    @classmethod
    def fetch(cls, ident) : 
        db = cls.get_db()
        
        try : 
            return db[ident]
        
        except TypeError : 
            if db is None : 
                msg = "database not set; call '{}.set_db(my_db)'"
                raise MissingDatabaseError(msg.format(cls.__name__))
            else : 
                raise
                
    def __repr__(self) : 
        if hasattr(self, 'serial') : 
            cls_name = self.__class__.__name__
            return '<{} serial={!r}>'.format(cls_name.serial)
        
        else : 
            return super().__repr__()
        


In [6]:
class Event(DbRecord) : 
    @property
    def venue(self) : 
        key = 'venue.{}'.format(self.venue_serial)
        return self.__class__.fetch(key)
    
    @property
    def speakers(self) :
        if not hasattr(self, '_speaker_objs') : 
            spkr_serials = self.__dict__['speakers']
            fetch = self.__class__.fetch
            self._speaker_objs = [fetch('speaker.{}'.format(key)) for key in spkr_serials]
            
        return self._speaker_objs
    
    def __repr__(self) : 
        if hasattr(self, 'name') : 
            cls_name = self.__class__.__name__
            return '<{} {!r}>'.format(cls_name, self.name)
        
        else : 
            return super().__repr__()
        
    

In [8]:
""" 
주문 항목 클래스
"""

class LineItem : 
    
    def __init__(self, description, weight, price) : 
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self) : 
        return self.weight * self.price
    


In [9]:
"""  
무게가 음수면 합계가 음수가됨 
"""

raisins = LineItem('Golden raisins', 10, 6.95)
raisins.subtotal()

69.5

In [10]:
raisins.weight = -20
raisins.subtotal()

-139.0

In [11]:
"""
검증하는 프로퍼티 : 데이터 속성, iv를 프로퍼티로 변경하기
"""

class LineItem : 
    
    def __init__(self, description, weight, price) : 
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self) : 
        return self.weight * self.price
    
    @property
    def weight(self) : 
        return self.__weight
    
    @weight.setter
    def weight(self, value) : 
        if value > 0 : 
            self.__weight = value
            
        else : 
            raise ValueError("value must be > 0")
            

In [12]:
raisins = LineItem('Golden raisins', 10, 6.95)
raisins.subtotal()

69.5

In [13]:
raisins.weight = -20
raisins.subtotal()

ValueError: value must be > 0

In [14]:
"""
위의 클래스와 동일 하지만, 데커레이터를 사용하지 않음
- 게터/세터 정의
- property 정의 및 공개 속성에 할당 
"""

class LineItem : 
    
    def __init__(self, description, weight, price) : 
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self) : 
        return self.weight * self.price
    

    def get_weight(self) : 
        return self.__weight
    

    def set_weight(self, value) : 
        if value > 0 : 
            self.__weight = value
            
        else : 
            raise ValueError("value must be > 0")
            
    weight = property(get_weight, set_weight)
    
    

In [15]:
raisins = LineItem('Golden raisins', 10, 6.95)
raisins.subtotal()

69.5

In [16]:
raisins.weight = -20
raisins.subtotal()

ValueError: value must be > 0

In [17]:
 """
 객체의 속성이 클래스 데이터 속성을 가르킴
 """
    
class Class : 
    data = 'the class data attr'
    
    @property
    def prop(self) : 
        return 'the prop value'
    
obj = Class()
vars(obj)

{}

In [18]:
obj.data

'the class data attr'

In [20]:
obj.data = 'bar'
vars(obj)

{'data': 'bar'}

In [21]:
Class.data

'the class data attr'

In [25]:
Class.prop

In [26]:
obj.prop

'the prop value'

In [27]:
obj.prop = 'foo'


AttributeError: can't set attribute

In [30]:
obj.__dict__['prop'] = 'foo'
vars(obj)

{'data': 'bar', 'prop': 'foo'}

In [31]:
obj.prop

'the prop value'

In [32]:
Class.prop = 'baz'

In [33]:
obj.prop

'foo'

In [35]:
def quantity(storage_name) : 
    """프로퍼티 팩토리"""
    
    
    def qty_getter(instance) : 
        return instance.__dict__[storage_name]
    
    def qty_setter(instance, value) : 
        if value > 0 : 
            instance.__dict__[storage_name] = value
            
        else : 
            raise ValueError('value must be > 0')
            
    return property(qty_getter, qty_setter)


class LineItem : 
    weight = quantity('weight')
    price = quantity('price')
    
    def __init__(self, description, weight, price) : 
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self) : 
        return self.weight * self.price

In [36]:
nutmeg = LineItem('Moluccan nutmeg', 8, 13.95)
nutmeg.weight, nutmeg.price

(8, 13.95)

In [37]:
sorted(vars(nutmeg).items())

[('description', 'Moluccan nutmeg'), ('price', 13.95), ('weight', 8)]

In [1]:
class Quantity : 
    
    def __init__(self, storage_name) : 
        self.storage_name = storage_name
        
    def __set__(self, instance, value) : 
        if value > 0 : 
            instance.__dict__[self.storage_name] = value
            
        else : 
            raise ValueError('value must be > 0')
            
class LineItem : 
    weight = Quantity('weight')
    price = Quantity('price')
    
    def __init__(self, description, weight, price) : 
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self) : 
        return self.weight * self.price
    
    

In [3]:
truffle = LineItem('White truffle', 100, 0)

ValueError: value must be > 0

In [8]:
class Quantity : 
    __counter = 0
    
    def __init__(self) : 
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter    
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter += 1
        
    def __get__(self, instance, owner) : 
        if instance is None : 
            return self
        
        else : 
            return getattr(instance, self.storage_name)
    
    def __set__(self, instance, value) :
        if value > 0:
            setattr(instance, self.storage_name, value)
            
        else : 
            raise ValueError('value must be > 0')
            
    
class LineItem : 
    weight = Quantity()
    price = Quantity()
    
    def __init__(self, description, weight, price) : 
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self) : 
        return self.weight * self.price
    
    
    

In [9]:
coconuts = LineItem('Brazilian coconut', 20, 17.95)
coconuts.weight, coconuts.price

(20, 17.95)

In [10]:
getattr(coconuts, '_Quantity#0'), getattr(coconuts, '_Quantity#1')

(20, 17.95)

In [11]:
import abc

class AutoStorage : 
    __counter = 0
    
    def __init__(self) : 
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter += 1
        
    def __get__(self, instance, owner) : 
        if instance is None : 
            return self
        
        else : 
            return getattr(instance, self.storage_name)
        
    
    def __set__(self, instance, value) : 
        setattr(instance, self.storage_name, value)
        
class Validated(abc.ABC, AutoStorage) : 
    
    def __set__(self, instance, value) :
        value = self.validate(instance, value)
        super().__set__(instance, value)
        
    @abc.abstractmethod
    def validate(self, instance, value) : 
        """검증된 값을 반환, ValueError 발생"""
        
class Quantity(Validated) : 
    """0 보다 큰 수"""
    
    def validate(self, instance, value) : 
        if value <= 0 :
            return ValueError('value must be > 0')
        
        return value
    

class NoneBlank(Validated) : 
    """최소 하나 이상의 비공백 문자가 들어 있는 문자열"""
    
    def validate(self, instance, value) : 
        value = value.strip()
        
        if len(value) == 0 : 
            raise ValueError('value cannot be empty or blank')
            
        return value

In [14]:
class LineItem : 
    description = NoneBlank()
    weight = Quantity()
    price = Quantity()
    
    def __init__(self, descriptiom, weight, price) : 
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self) : 
        return self.weight * self.price
    
    

In [15]:
### 단지 출력하기 위한 보조 함수 ###

def cls_name(obj_or_cls) : 
    cls = type(obj_or_cls)
    
    if cls is type : 
        cls = obj_or_cls
    
    return cls.__name__.split('.')[-1]

def display(obj) : 
    cls = type(obj)
    
    if cls is type : 
        return '<class {}>'.format(obj.__name__)
    
    elif cls in [type(None), int] : 
        return repr(obj)
    
    else : 
        return '<{} object>'.format(cls_name(obj))

def print_args(name, *args) :
    pseudo_args = ', '.join(display(x) for x in args) 
    print('-> {}.__{}__({})'.format(cls_name(args[0]), name, pseudo_args))
    
    
class Overriding : 
    """데이터 디스크립터 혹은 강제 디스크립터라고함"""
    
    def __get__(self, instance, owner) :
        print_args('get', self, instance, owner)
        
    def __set__(self, instance, value) : 
        print_args('set', self, instance, value)
        
class OverridingNoGet : 
    """``__get__()``이 없는 오버라이딩 디스크립터"""
    
    def __set__(self, instance, value) : 
        print_args('set', self, instance, value)
        
class NonOverriding : 
    """비데이터 디스크립터 혹은 가릴 수 있는 디스크립터라고함"""
    
    def __get__(self, instance, owner) : 
        print_args('get', self, instance, owner)
        
class Managed : 
    over = Overriding()
    over_no_get = OverridingNoGet()
    non_over = NonOverriding()
    
    def spam(self) : 
        print('-> Managed.spam({})'.format(display(self)))

In [16]:
obj = Managed()

In [17]:
obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [18]:
Managed.over

-> Overriding.__get__(<Overriding object>, None, <class Managed>)


In [19]:
obj.over = 7

-> Overriding.__set__(<Overriding object>, <Managed object>, 7)


In [20]:
obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [21]:
obj.__dict__['over'] = 8
vars(obj)

{'over': 8}

In [22]:
obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)
